In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import h2o
from h2o.automl import H2OAutoML
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.6" 2020-01-14; OpenJDK Runtime Environment (build 11.0.6+10-post-Ubuntu-1ubuntu118.04.1); OpenJDK 64-Bit Server VM (build 11.0.6+10-post-Ubuntu-1ubuntu118.04.1, mixed mode, sharing)
  Starting server from /home/edu/.local/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp031p70qc
  JVM stdout: /tmp/tmp031p70qc/h2o_edu_started_from_python.out
  JVM stderr: /tmp/tmp031p70qc/h2o_edu_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,01 secs
H2O cluster timezone:,Europe/Paris
H2O data parsing timezone:,UTC
H2O cluster version:,3.28.0.3
H2O cluster version age:,25 days
H2O cluster name:,H2O_from_python_edu_yuaa4k
H2O cluster total nodes:,1
H2O cluster free memory:,1.887 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"


In [2]:
train_high = h2o.import_file('../cleaned-data/05-train_high-catAsDummies-withXYZ.csv')
train_low = h2o.import_file('../cleaned-data/05-train_low-catAsDummies-withXYZ.csv')
test_high = h2o.import_file('../cleaned-data/05-test_high-catAsDummies-withXYZ.csv')
test_low = h2o.import_file('../cleaned-data/05-test_low-catAsDummies-withXYZ.csv')      
train_high.head()

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


id,carat,depth,table,x,y,z,cut_0,cut_1,cut_2,cut_3,cut_4,color_1,color_2,color_3,color_4,color_5,color_6,color_7,clarity_0,clarity_1,clarity_2,clarity_3,clarity_4,clarity_5,clarity_6,clarity_7,price
8,2.05,62.4,57,8.17,8.09,5.08,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,18193
34,2.01,64.9,60,7.82,7.78,5.06,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,13744
42,2,64.2,52,7.91,7.88,5.07,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,11400
72,2,60.3,56,8.08,8.11,4.88,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,16425
82,2.28,62.7,57,8.43,8.36,5.26,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,16450
93,2.02,60.4,58,8.2,8.16,4.94,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,17357
141,2,60.7,60,8.07,8.11,4.91,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,15312
154,2.01,61.5,62,8.07,7.94,4.92,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,15475
165,2.31,62.9,58,8.43,8.34,5.27,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,17218
167,2.01,60.6,57,8.19,8.09,4.92,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,17383


In [3]:
list(map(lambda x: print(x.shape),[train_high, train_low, test_high, test_low]))

(1615, 28)
(38730, 28)
(521, 27)
(12928, 27)


[None, None, None, None]

In [4]:
xTrainCols = train_high.col_names[1:-1]
xTrainCols

['carat',
 'depth',
 'table',
 'x',
 'y',
 'z',
 'cut_0',
 'cut_1',
 'cut_2',
 'cut_3',
 'cut_4',
 'color_1',
 'color_2',
 'color_3',
 'color_4',
 'color_5',
 'color_6',
 'color_7',
 'clarity_0',
 'clarity_1',
 'clarity_2',
 'clarity_3',
 'clarity_4',
 'clarity_5',
 'clarity_6',
 'clarity_7']

In [5]:
train_frame = train_high
model_name = 'diamonds_high'

aml_high = H2OAutoML(max_runtime_secs = 300, project_name = f'{model_name}', keep_cross_validation_models=True)
aml_high.train(x=xTrainCols, y = 'price', training_frame = train_frame)
pred_high = aml_high.leader.predict(test_high)
display(pred_high)
display(aml_high.leaderboard)

AutoML progress: |████████████████████████████████████████████████████████| 100%
stackedensemble prediction progress: |████████████████████████████████████| 100%


predict
16545.5
16282.2
16019.2
16665.1
15522.1
13044.3
16875.9
16868.8
14732.8
16330.9


model_id,mean_residual_deviance,rmse,mse,mae,rmsle
StackedEnsemble_BestOfFamily_AutoML_20200302_095451,2.77492e+06,1665.81,2.77492e+06,1295.86,0.118979
StackedEnsemble_AllModels_AutoML_20200302_095451,2.7786e+06,1666.91,2.7786e+06,1295.38,0.119127
XGBoost_1_AutoML_20200302_095451,2.80204e+06,1673.93,2.80204e+06,1319.05,0.120058
XGBoost_grid__1_AutoML_20200302_095451_model_2,2.83379e+06,1683.39,2.83379e+06,1326.08,0.120946
XGBoost_grid__1_AutoML_20200302_095451_model_3,2.83793e+06,1684.62,2.83793e+06,1326.07,0.121122
XGBoost_2_AutoML_20200302_095451,2.8446e+06,1686.59,2.8446e+06,1322.08,0.120807
XGBoost_3_AutoML_20200302_095451,2.85922e+06,1690.92,2.85922e+06,1339.77,0.121469
GBM_4_AutoML_20200302_095451,2.85974e+06,1691.08,2.85974e+06,1323.8,0.121413
XGBoost_grid__1_AutoML_20200302_095451_model_4,2.87104e+06,1694.41,2.87104e+06,1317.51,0.121331
GBM_2_AutoML_20200302_095451,2.87277e+06,1694.92,2.87277e+06,1323.63,0.122075


In [6]:
tableHighPreds = test_high['id'].cbind(pred_high)
tableHighPreds = tableHighPreds.rename(columns={'predict':'price'})

In [7]:
train_frame = train_low
model_name = 'diamonds_low'

aml_low = H2OAutoML(max_runtime_secs = 300, project_name = f'{model_name}', keep_cross_validation_models=True)
aml_low.train(x=xTrainCols, y = 'price', training_frame = train_frame)
pred_low = aml_low.leader.predict(test_low)
display(pred_low)
display(aml_low.leaderboard)

AutoML progress: |████████████████████████████████████████████████████████| 100%
stackedensemble prediction progress: |████████████████████████████████████| 100%


predict
4426.16
1023.57
7077.31
1470.5
2224.82
1939.41
1815.82
3491.33
15099.8
3686.54


model_id,mean_residual_deviance,rmse,mse,mae,rmsle
StackedEnsemble_AllModels_AutoML_20200302_095957,183477,428.342,183477,225.514,0.0900198
StackedEnsemble_BestOfFamily_AutoML_20200302_095957,184253,429.247,184253,227.276,0.0916806
XGBoost_1_AutoML_20200302_095957,185430,430.616,185430,229.266,0.0933194
XGBoost_2_AutoML_20200302_095957,192142,438.34,192142,227.848,0.0885417
DRF_1_AutoML_20200302_095957,215090,463.778,215090,245.752,0.0980872
XGBoost_3_AutoML_20200302_095957,237756,487.603,237756,271.153,0.119445
GBM_1_AutoML_20200302_095957,402840,634.697,402840,370.03,0.190534
GBM_2_AutoML_20200302_095957,2.16187e+06,1470.33,2.16187e+06,1053.41,0.577914
GBM_3_AutoML_20200302_095957,8.18346e+06,2860.67,8.18346e+06,2189.3,0.948307
GLM_1_AutoML_20200302_095957,1.11047e+07,3332.36,1.11047e+07,2572.5,1.05989


In [8]:
tableLowPreds = test_low['id'].cbind(pred_low)
tableLowPreds = tableLowPreds.rename(columns={'predict':'price'})

In [9]:
combinedTable = tableLowPreds.concat(tableHighPreds, axis=0)
combinedTable.shape

(13449, 2)

In [11]:
h2o.export_file(frame=combinedTable,path='../predictions/14-Auto-ML-Split-data-Dummies-compl-data-v2.csv')

Export File progress: |███████████████████████████████████████████████████| 100%
